In [5]:
from elasticsearch import Elasticsearch

# Stap 1: Verbinding maken met Elasticsearch
es = Elasticsearch("http://localhost:9200")  # Update indien nodig met je Elasticsearch URL

# Stap 2: Definieer de Aggregatie-Query met geohash
query = {
    "size": 0,
    "aggs": {
        "location_buckets": {
            "geohash_grid": {
                "field": "location",
                "precision": 5  # Past de locatie aan tot ongeveer 100 meter precisie
            },
            "aggs": {
                "time_buckets": {
                    "date_histogram": {
                        "field": "timestamp",
                        "fixed_interval": "5m"
                    },
                    "aggs": {
                        "average_measurement": {
                            "terms": {
                                "field": "measurement_type.keyword"
                            },
                            "aggs": {
                                "average_value": {
                                    "avg": {
                                        "field": "measurement_value"
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

# Stap 3: Voer de Query uit
response = es.search(index="velo_measurements", body=query)

# Stap 4: Verwerk en toon de resultaten
for location in response['aggregations']['location_buckets']['buckets']:
    location_key = location['key']
    print(f"Location Zone (GeoHash): {location_key}")

    for time_bucket in location['time_buckets']['buckets']:
        timestamp = time_bucket['key_as_string']
        print(f"  Time Interval: {timestamp}")

        for measurement in time_bucket['average_measurement']['buckets']:
            measurement_type = measurement['key']
            average_value = measurement['average_value']['value']
            print(f"    Measurement: {measurement_type}, Average: {average_value}")


Location Zone (GeoHash): u155k
  Time Interval: 2024-10-30T19:40:00.000Z
  Time Interval: 2024-10-30T19:45:00.000Z
  Time Interval: 2024-10-30T19:50:00.000Z
  Time Interval: 2024-10-30T19:55:00.000Z
  Time Interval: 2024-10-30T20:00:00.000Z
  Time Interval: 2024-10-30T20:05:00.000Z
  Time Interval: 2024-10-30T20:10:00.000Z
  Time Interval: 2024-10-30T20:15:00.000Z
Location Zone (GeoHash): u1557
  Time Interval: 2024-10-30T19:40:00.000Z
  Time Interval: 2024-10-30T19:45:00.000Z
  Time Interval: 2024-10-30T19:50:00.000Z
  Time Interval: 2024-10-30T19:55:00.000Z
  Time Interval: 2024-10-30T20:00:00.000Z
  Time Interval: 2024-10-30T20:05:00.000Z
